
<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This Runbook demonstrates k8s: Get kube system config map using unSkript legos.</b>
</div>

<br>

<center><h2>k8s: Get kube system config map</h2></center>

# Steps Overview
    1) Get k8s kube system config map.
    2) Post the all k8s kube system config maps to slack channel.

Here we will use unSkript Get k8s kube system config map Lego. This lego takes config_map_name and namespace as input. These inputs are used to Get k8s kube system config map.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from typing import List

from kubernetes import client
from pydantic import BaseModel, Field
from tabulate import tabulate


from beartype import beartype
@beartype
def k8s_get_config_map_kube_system(handle, config_map_name: str = '', namespace: str = 'all') -> List:

    coreApiClient = client.CoreV1Api(api_client=handle)

    res = coreApiClient.list_namespaced_config_map(
        namespace=namespace, pretty=True)
    if len(res.items) > 0:
        if config_map_name:
            config_maps = list(
                filter(lambda x: (x.metadata.name == config_map_name), res.items))
        else:
            config_maps = res.items
        for config in config_maps:
            data_set_1 = []
            data_set_1.append("Name:")
            data_set_1.append(config.metadata.name)

            data_set_2 = []
            data_set_2.append("Namespace:")
            data_set_2.append(config.metadata.namespace)

            data_set_3 = []
            data_set_3.append("Labels:")
            data_set_3.append(config.metadata.labels)

            data_set_4 = []
            data_set_4.append("Annotations:")
            data_set_4.append(config.metadata.annotations)

            data_set_5 = []
            data_set_5.append("Data:")
            data_set_5.append(config.data)

            tabular_config_map = []
            tabular_config_map.append(data_set_1)
            tabular_config_map.append(data_set_2)
            tabular_config_map.append(data_set_3)
            tabular_config_map.append(data_set_4)
            tabular_config_map.append(data_set_5)

            print(tabulate(tabular_config_map, tablefmt="github"))

        return config_maps
    return None


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "namespace": "namespace"
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(k8s_get_config_map_kube_system, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

if hasattr(task, 'output'):
    if isinstance(task.output, (list, tuple)):
        for item in task.output:
            print(f'item: {item}')
    elif isinstance(task.output, dict):
        for item in task.output.items():
            print(f'item: {item}')
    else:
        print(f'Output for {task.name}')
        print(task.output)
    w.tasks[task.name]= task.output

Here we will use unSkript Post Slack Message Lego. This lego takes channel: str and message: str as input. These inputs are used to post the message to the specified Slack channel.

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)


from beartype import beartype
def legoPrinter(func):
    def Printer(*args, **kwargs):
        output = func(*args, **kwargs)
        if output:
            channel = kwargs["channel"]
            pp.pprint(print(f"Message sent to Slack channel {channel}"))
        return output
    return Printer


@legoPrinter
@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> bool:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return True
    except SlackApiError as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        return False
    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return False


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "channel": "channel",
    "message": "f\\"Config map for namespace:{namespace}: {cfg_map}\\""
    }''')

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.output = task.execute(slack_post_message, hdl=hdl, args=args)
    if task.output_name != None:
        globals().update({task.output_name: task.output[0]})

### Conclusion
In this Runbook, we demonstrated the use of unSkript's k8s legos to run k8s action and fetches the kube system config map for a k8s cluster and posts to a slack channel. To view the full platform capabilities of unSkript please visit https://unskript.com